In [1]:
import pandas as pd
import numpy as np
from tqdm import tqdm
tqdm.pandas()

In [2]:
df_keywords_titles = pd.read_json('/mnt/hdd01/patentsview/Similarity Search - CPC Classification and Claims/cpc_yake_keywords_list_noun_chunks_embeddings.json')

# Convert columns 'keyword_patentsberta_embedding', 'keyword_climatebert_embedding', 'keyword_bertforpatents_embedding' to numpy arrays
df_keywords_titles['keyword_yake_patentsberta_embedding'] = df_keywords_titles['keyword_yake_patentsberta_embedding'].progress_apply(lambda x: np.array(x, dtype=np.float32))
df_keywords_titles['keyword_yake_climatebert_embedding'] = df_keywords_titles['keyword_yake_climatebert_embedding'].progress_apply(lambda x: np.array(x, dtype=np.float32))
df_keywords_titles['keyword_yake_bertforpatents_embedding'] = df_keywords_titles['keyword_yake_bertforpatents_embedding'].progress_apply(lambda x: np.array(x, dtype=np.float32))

100%|██████████| 418/418 [00:00<00:00, 24515.06it/s]


In [3]:
df_keywords_claims = pd.read_json('/mnt/hdd01/patentsview/Similarity Search - CPC Classification and Claims/df_keywords_list_agg_uspto_epo_rel_embeddings_noun_chunks.json')

# Convert columns 'keyword_yake_patentsberta_embedding', 'keyword_yake_climatebert_embedding', 'keyword_yake_bertforpatents_embedding' to numpy arrays
df_keywords_claims['keyword_yake_patentsberta_embedding'] = df_keywords_claims['keyword_yake_patentsberta_embedding'].progress_apply(lambda x: np.array(x, dtype=np.float32))
df_keywords_claims['keyword_yake_climatebert_embedding'] = df_keywords_claims['keyword_yake_climatebert_embedding'].progress_apply(lambda x: np.array(x, dtype=np.float32))
df_keywords_claims['keyword_yake_bertforpatents_embedding'] = df_keywords_claims['keyword_yake_bertforpatents_embedding'].progress_apply(lambda x: np.array(x, dtype=np.float32))

100%|██████████| 170922/170922 [00:07<00:00, 24054.68it/s]


# Testing

In [ ]:
# Print out all keyword_yake_lemma for cpc_class_symbol = Y02W
print(df_keywords_titles[df_keywords_titles['cpc_class_symbol'].progress_apply(lambda x: 'Y02W10/33' in x)][['keyword_yake_lemma', 'cpc_class_symbol']])

In [ ]:
# Set search test to row with keyword = solar cells
df_search_test_titles = df_keywords_titles[df_keywords_titles['keyword_yake_lemma'] == 'wind energy']

In [ ]:
# Reset index of df_search_test_titles
df_search_test_titles.reset_index(drop=True, inplace=True)

In [ ]:
df_search_test_titles

# Similarity Search

In [4]:
import faiss
from sklearn.preprocessing import normalize
from sklearn.neighbors import NearestNeighbors

# Euclidean Distance - kNN

In [ ]:
# Eucledian distance
index_bertforpatents = faiss.IndexFlatL2(1024)   # build the index
index_climatebert = faiss.IndexFlatL2(768)   # build the index
index_patentsberta = faiss.IndexFlatL2(768)   # build the index

# Add df_keywords_claims column 'keyword_yake_bertforpatents_embedding' to index
index_bertforpatents.add(np.array(df_keywords_claims['keyword_yake_bertforpatents_embedding'].tolist()))
index_climatebert.add(np.array(df_keywords_claims['keyword_yake_climatebert_embedding'].tolist()))
index_patentsberta.add(np.array(df_keywords_claims['keyword_yake_patentsberta_embedding'].tolist()))

query_vector_bertforpatents = np.array(df_search_test_titles['keyword_bertforpatents_embedding'][0]).reshape(1, -1)
query_vector_climatebert = np.array(df_search_test_titles['keyword_climatebert_embedding'][0]).reshape(1, -1)
query_vector_patentsberta = np.array(df_search_test_titles['keyword_patentsberta_embedding'][0]).reshape(1, -1)

In [ ]:
k = 25
D_bertforpatents, I_bertforpatents = index_bertforpatents.search(query_vector_bertforpatents, k)
D_climatebert, I_climatebert = index_climatebert.search(query_vector_climatebert, k)
D_patentsberta, I_patentsberta = index_patentsberta.search(query_vector_patentsberta, k)

In [ ]:
# Print row of df_keywords_claims that match the index
df_keywords_claims.iloc[I_patentsberta[0]]

# Cosine Similarity - kNN

In [ ]:
# Cosine similarity
index_bertforpatents = faiss.index_factory(1024, "Flat", faiss.METRIC_INNER_PRODUCT)
index_climatebert = faiss.index_factory(768, "Flat", faiss.METRIC_INNER_PRODUCT)
index_patentsberta = faiss.index_factory(768, "Flat", faiss.METRIC_INNER_PRODUCT)

faiss.normalize_L2(np.array(df_keywords_claims['keyword_yake_bertforpatents_embedding'].tolist()))
faiss.normalize_L2(np.array(df_keywords_claims['keyword_yake_climatebert_embedding'].tolist()))
faiss.normalize_L2(np.array(df_keywords_claims['keyword_yake_patentsberta_embedding'].tolist()))

index_bertforpatents.add(np.array(df_keywords_claims['keyword_yake_bertforpatents_embedding'].tolist()))
index_climatebert.add(np.array(df_keywords_claims['keyword_yake_climatebert_embedding'].tolist()))
index_patentsberta.add(np.array(df_keywords_claims['keyword_yake_patentsberta_embedding'].tolist()))

query_vector_bertforpatents = np.array(df_search_test_titles['keyword_yake_bertforpatents_embedding'][0]).reshape(1, -1)
query_vector_climatebert = np.array(df_search_test_titles['keyword_yake_climatebert_embedding'][0]).reshape(1, -1)
query_vector_patentsberta = np.array(df_search_test_titles['keyword_yake_patentsberta_embedding'][0]).reshape(1, -1)

faiss.normalize_L2(query_vector_bertforpatents)
faiss.normalize_L2(query_vector_climatebert)
faiss.normalize_L2(query_vector_patentsberta)

In [ ]:
k = 6
D_bertforpatents, I_bertforpatents = index_bertforpatents.search(query_vector_bertforpatents, k)
D_climatebert, I_climatebert = index_climatebert.search(query_vector_climatebert, k)
D_patentsberta, I_patentsberta = index_patentsberta.search(query_vector_patentsberta, k)

In [ ]:
# Print row of df_keywords_claims that match the index
df_keywords_claims.iloc[I_bertforpatents[0]]

# Cosine Similarity - Margin

In [21]:
# Convert lists of embeddings to a 2D array
bertforpatents_embeddings = np.vstack(df_keywords_claims['keyword_yake_bertforpatents_embedding'].apply(np.array))
climatebert_embeddings = np.vstack(df_keywords_claims['keyword_yake_climatebert_embedding'].apply(np.array))
patentsberta_embeddings = np.vstack(df_keywords_claims['keyword_yake_patentsberta_embedding'].apply(np.array))

# Normalize embeddings
# bertforpatents_embeddings_normalized = normalize(bertforpatents_embeddings)
# climatebert_embeddings_normalized = normalize(climatebert_embeddings)
# patentsberta_embeddings_normalized = normalize(patentsberta_embeddings)

# Initialize NearestNeighbors with radius
radius = 0.25
nn_bertforpatents = NearestNeighbors(radius=radius, metric='cosine')
# nn_climatebert = NearestNeighbors(radius=radius, metric='cosine')
# nn_patentsberta = NearestNeighbors(radius=radius, metric='cosine')

# Fit the models with normalized embeddings
nn_bertforpatents.fit(bertforpatents_embeddings)
# nn_climatebert.fit(climatebert_embeddings)
# nn_patentsberta.fit(patentsberta_embeddings)

NearestNeighbors(metric='cosine', radius=0.25)

In [22]:
df_keywords_titles.head()

,keyword_yake_lemma,yake_conf_score,cpc_class_symbol,keyword_yake_patentsberta_embedding,keyword_yake_climatebert_embedding,keyword_yake_bertforpatents_embedding,neighbors_keyword_yake_bertforpatents_embedding,keywords_keyword_yake_bertforpatents_embedding
0,abiotic stress,0.025709,[Y02A40/13],"[-0.39456344, -0.76772654, -0.38135096, 0.1383...","[-0.014673066, 0.11892948, 0.0026377595, -0.01...","[0.10681395, -0.6905355, 0.47798496, 0.0765340...","[73, 74, 75, 76, 42393, 42394, 42395]","[abiotic stress, abiotic stress condition, abi..."
1,absorption based system,0.008321,[Y02B30/62],"[-0.11752405, -0.7287777, -0.1876528, 0.013015...","[0.010742915, 0.08329072, -0.025938502, -0.048...","[-0.38561568, 0.82870007, 0.39117342, -0.29798...",[],[]
2,acid,0.216929,"[Y02P20/30, Y02W30/74]","[0.1558558, -0.46968195, -0.38458294, 0.150290...","[-0.03934484, 0.110011734, 0.018712386, 0.0044...","[-0.49631354, -0.1980652, 0.2423975, -0.069928...","[1123, 1132, 1139, 1188, 1207, 1237, 1282, 130...","[acid amount, acid and base, acid aqueous, aci..."
3,acid or caprolactam,0.097004,[Y02P20/30],"[0.06250761, -0.70466757, -0.32794353, 0.00913...","[0.0009763084, 0.081545256, 0.042667516, -0.02...","[-0.40428126, 0.24819237, 0.43703473, -0.73723...",[],[]
4,active power filtering,0.008321,[Y02E40/20],"[-0.20968652, -0.887124, -0.28240728, -0.10112...","[0.022758827, 0.12130108, -0.036426675, 0.0244...","[-0.75558186, 0.27234048, -0.7379539, -1.25931...",[2062],[active power filter]


In [23]:
# Define the embeddings to process
# embeddings = ['keyword_yake_bertforpatents_embedding', 'keyword_yake_climatebert_embedding', 'keyword_yake_patentsberta_embedding']
embeddings = ['keyword_yake_bertforpatents_embedding']
# nn_models = [nn_bertforpatents, nn_climatebert, nn_patentsberta]
nn_models = [nn_bertforpatents]
k_neighbors = 100  # Maximum number of neighbors to return

def get_keywords_by_indices(indices, df_source, k_neighbors=k_neighbors):
    return df_source.iloc[indices[:k_neighbors]]['keyword_yake_lemma'].tolist() if len(indices) else []

def process_embeddings(row, nn_model, df_keywords_claims, k_neighbors=k_neighbors):
    # query_vector = normalize(np.array([row]).reshape(1, -1))
    query_vector = np.array([row]).reshape(1, -1)
    distances, indices = nn_model.radius_neighbors(query_vector)
    neighbors = indices[0][:k_neighbors] if indices[0].size else np.array([])
    keywords = get_keywords_by_indices(neighbors, df_keywords_claims, k_neighbors)
    return neighbors.tolist(), keywords

# Process each embedding
for embedding, nn_model in zip(embeddings, nn_models):
    neighbors_col = f'neighbors_{embedding}'
    keywords_col = f'keywords_{embedding}'

    results = df_keywords_titles[embedding].progress_apply(lambda x: process_embeddings(x, nn_model, df_keywords_claims, k_neighbors))
    df_keywords_titles[neighbors_col] = results.progress_apply(lambda x: x[0])
    df_keywords_titles[keywords_col] = results.progress_apply(lambda x: x[1])

100%|██████████| 418/418 [00:00<00:00, 825432.71it/s]


In [24]:
df_keywords_titles.to_json(f"/mnt/hdd01/patentsview/Similarity Search - CPC Classification and Claims/Similarity Search/df_keyword_titles_cosine_similarity_radius_{str(radius).replace('.','')}_neighbors_{str(k_neighbors)}_noun_chunks.json", orient='records')